# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [2]:
#Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [12]:
#Your code here
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [4]:
viewers = set(df[df.action == 'view']['id'].unique())
clickers = set(df[df.action == 'click']['id'].unique())

num_viewers = len(viewers)
num_clickers = len(clickers)
num_viewers_no_click = len(viewers - clickers)
num_clickers_no_view = len(clickers - viewers)

print(f"Number of viewers: {num_viewers}\tNumber of clickers: {num_clickers}")
print(f"Number of viewers who didn't click: {num_viewers_no_click}")
print(f"Number of clickers who didn't view: {num_clickers_no_view}")


Number of viewers: 6328	Number of clickers: 1860
Number of viewers who didn't click: 4468
Number of clickers who didn't view: 0


In [14]:
# Convert clicks into a binary variable on a user-by-user basis for control group
control = df[df['group'] == 'control'].pivot_table(index='id', columns='action', aggfunc='size', fill_value=0)

# Convert clicks into a binary variable on a user-by-user basis for experiment group
experiment = df[df['group'] == 'experiment'].pivot_table(index='id', columns='action', aggfunc='size', fill_value=0)

# Fill NaN values with 0
control = control.fillna(0)
experiment = experiment.fillna(0)

# Display sample sizes, total clicks, and average click rates
control_size = len(control)
experiment_size = len(experiment)
control_total_clicks = control['click'].sum()
experiment_total_clicks = experiment['click'].sum()
control_avg_click_rate = control['click'].mean()
experiment_avg_click_rate = experiment['click'].mean()

print(f"Sample sizes:\tControl: {control_size}\tExperiment: {experiment_size}")
print(f"Total Clicks:\tControl: {control_total_clicks}\tExperiment: {experiment_total_clicks}")
print(f"Average click rate:\tControl: {control_avg_click_rate}\tExperiment: {experiment_avg_click_rate}")


Sample sizes:	Control: 3332	Experiment: 2996
Total Clicks:	Control: 932	Experiment: 928
Average click rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


In [6]:
eids = set(df[df.group=='experiment']['id'].unique())
cids = set(df[df.group=='control']['id'].unique())
print('Overlap of experiment and control groups: {}'.format(len(eids&cids)))

Overlap of experiment and control groups: 0


In [8]:
import numpy as np
import scipy.stats as stats

def welch_t(a, b):
    
    

    numerator = a.mean() - b.mean()
    
    # “ddof = Delta Degrees of Freedom”: the divisor used in the calculation is N - ddof, 
    #  where N represents the number of elements. By default ddof is zero.
    
    denominator = np.sqrt(a.var(ddof=1)/a.size + b.var(ddof=1)/b.size)
    
    return np.abs(numerator/denominator)

def welch_df(a, b):
    

    
    s1 = a.var(ddof=1) 
    s2 = b.var(ddof=1)
    n1 = a.size
    n2 = b.size
    
    numerator = (s1/n1 + s2/n2)**2
    denominator = (s1/ n1)**2/(n1 - 1) + (s2/ n2)**2/(n2 - 1)
    
    return numerator/denominator


def p_value_welch_ttest(a, b, two_sided=False):

    t = welch_t(a, b)
    df = welch_df(a, b)
    
    p = 1-stats.t.cdf(np.abs(t), df)
    
    if two_sided:
        return 2*p
    else:
        return p

In [15]:
p_value_welch_ttest(control.click, experiment.click)

0.004466402814337078

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [17]:
control_rate = control.click.mean()
expected_experiment_clicks_under_null = control_rate * len(experiment)
print(expected_experiment_clicks_under_null)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [20]:
#Your code here
n = len(experiment)
p = control_rate
var = n * p * (1-p)
std = np.sqrt(var)
print(std)
actual_experiment_clicks = experiment.click.sum()
z_score = (actual_experiment_clicks - expected_experiment_clicks_under_null)/std
print(z_score)

24.568547907005815
3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [21]:
#Your code here
import scipy.stats as stats
p_val = stats.norm.sf(z_score) #or 1 - stats.norm.cdf(z_score)
print(p_val)

0.00012486528006951198


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.